<a href="https://colab.research.google.com/github/William-Burbano-Lima/Ciberseguridad_explorador/blob/main/5S1DL1_LABORATORIO_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Talento Tech Ciberseguridad

Explorador
___
# **PRÁCTICA DE LABORATORIO 01**

Identificar **paquetes e IPs sospechosas** en una captura de tráfico de red con **Wireshark** es un proceso clave en auditoría de seguridad. Para hacerlo correctamente, es necesario tener un enfoque estructurado, basado en patrones anómalos de tráfico, protocolos inusuales o conexiones a direcciones desconocidas.

Aquí te dejo una guía para identificar estos paquetes e IPs sospechosas:

---

## **1. Análisis Visual de la Captura**
Cuando inicias la captura en Wireshark, los paquetes aparecerán en tiempo real. Algunos indicadores que pueden señalar tráfico sospechoso incluyen:

- **Frecuencia Anormal de Paquetes:**
  Si ves un **gran volumen de tráfico** desde o hacia una sola IP en un corto período de tiempo, esto podría indicar exfiltración de datos, ataque DDoS o un malware.
  
- **Direcciones IP Externas Desconocidas:**
  Si una dirección IP interna se conecta frecuentemente a direcciones **exteriores** (fuera de la red de tu empresa), podría ser un intento de fuga de datos.
  
- **Protocolos Inusuales:**
  El tráfico que utiliza protocolos poco comunes (como **FTP, SSH, Telnet, RDP**) podría ser sospechoso, especialmente si no están autorizados o no deberían ser utilizados en tu red.

---

## **2. Filtrar por IPs Sospechosas**
### **Filtrar tráfico hacia una IP específica**
Supón que sospechas de una IP externa (`192.168.1.100`), puedes utilizar el siguiente filtro para **aislar todo el tráfico hacia y desde esa IP**:
```
ip.addr == 192.168.1.100
```
- Si ves una cantidad inusualmente alta de tráfico hacia esta IP, puede ser indicativo de una **actividad sospechosa**.

### **Filtrar tráfico externo hacia tu red interna**
Si ves paquetes que se dirigen **fuera de tu red interna**, como a una IP desconocida, filtra así:
```
ip.dst != 192.168.1.0/24
```
*(Asumiendo que la red interna tiene el rango `192.168.1.x`)*  
Esto te mostrará todo el tráfico dirigido a direcciones fuera de tu subred interna.

### **Filtrar tráfico hacia puertos no autorizados**
Si ves paquetes con puertos como **FTP (21)**, **SSH (22)** o **Telnet (23)**, que no están permitidos en la red, puedes filtrarlos de la siguiente manera:
```
tcp.port == 21
```
Esto puede ayudarte a detectar tráfico no autorizado en puertos típicos de servicios de acceso remoto.

---

## **3. Inspección de Protocolos Sospechosos**
### **Paquetes HTTP no esperados o maliciosos**
Los **POST requests** en **HTTP** pueden ser un intento de enviar datos a un servidor malicioso. Para encontrar este tipo de tráfico:
```
http.request.method == "POST"
```
Si ves muchas peticiones **POST** a servidores que no reconoces, podría ser una señal de **exfiltración de datos**.

### **Tráfico HTTPS sospechoso**
El tráfico **HTTPS** es cifrado, por lo que no puedes ver el contenido, pero aún puedes identificar las **conexiones hacia sitios no esperados**:
```
tls
```
Si hay conexiones a **dominios extraños** o a sitios no aprobados por la organización, esto podría indicar comunicación con un servidor malicioso.

### **Tráfico ICMP (ping) o Tráfico de Escaneo de Red**
El tráfico **ICMP** inusualmente alto o tráfico hacia un rango de IPs desconocido puede ser una señal de un **ataque de escaneo de red**.
```
icmp
```
Los ataques de **ping flood** o **escaneo de puertos** utilizan ICMP para determinar la disponibilidad de los hosts en la red.

---

## **4. Inspección de DNS**
Si hay tráfico DNS **hacia dominios sospechosos** o **de consulta excesiva** a servidores de DNS no autorizados, es un **indicador claro** de actividad maliciosa. Filtra las consultas DNS con:
```
dns.qry.name contains "suspicious-domain.com"
```
- Esto te ayudará a identificar dominios **que no forman parte de tu infraestructura autorizada**.

---

## **5. Inspección de Paquetes de Datos Grandes o Comprimidos**
Los paquetes de gran tamaño (especialmente en protocolos como **FTP** o **HTTP**), pueden indicar que se están enviando grandes cantidades de datos fuera de la red.

Filtra paquetes grandes con:
```
frame.len > 1000
```
Si identificas un flujo de datos grandes hacia una IP externa, podría ser una **fuga de información**.

---

## **6. Análisis de Flujos de Comunicación (TCP Stream)**
A veces, los paquetes aislados no son tan útiles sin un contexto. Para obtener el flujo completo de comunicación entre dos IPs, puedes usar la opción **Follow TCP Stream**.

1. Haz clic derecho sobre cualquier paquete de **TCP**.
2. Selecciona **Follow > TCP Stream**.
3. Esto te dará una vista completa de la conversación entre las dos direcciones IP, permitiéndote analizar si hay **transacciones sospechosas**.

---

## **7. Revisión de Logs y Relaciones entre Paquetes**
Si identificas un paquete sospechoso, investiga su **relación con otros paquetes** en el flujo de tráfico:

- ¿Está asociado con **inyección de comandos** o comportamientos inusuales?
- ¿Hay paquetes de **respuestas inesperadas** (p. ej., servidores desconocidos respondiendo a solicitudes)?
  
---

## **Ejemplo Práctico:**
**Escenario:** Se ha detectado que una IP interna está enviando grandes cantidades de datos a un servidor externo no autorizado.

1. **Captura de tráfico**: Comienza capturando paquetes con Wireshark.
2. **Filtra por la IP interna**:  
   ```
   ip.addr == 192.168.1.100
   ```
3. **Analiza los protocolos**: Filtra por el tráfico de **HTTP/HTTPS** y **FTP**. Si encuentras **POST requests** o paquetes de gran tamaño a un servidor fuera de la red, puedes estar ante un intento de exfiltración de datos.
4. **Revisa el flujo de TCP**: Haz un **Follow TCP Stream** para ver si hay algo extraño en la comunicación.
5. **Revisa los DNS**: Filtra consultas de DNS a dominios sospechosos.

---

## **Conclusión**
Identificar paquetes e IPs sospechosas requiere una atención cuidadosa a patrones inusuales en el tráfico de red. Wireshark es una herramienta poderosa para realizar estas auditorías y, con los filtros adecuados, puedes detectar actividad maliciosa antes de que cause daño.  

___
William Burbano Lima